In [1]:
from os import listdir
from os.path import isfile, join
import pandas as pd
import openpyxl
from openpyxl import load_workbook
import xlwings
import xlwings as xl
import win32com.client
import time 

In [10]:
# Get All file names in a folder
mypath = r'C:\Pam_card\summary'
user_text = "C:\Pam_card\summary\file\_Data_CutOff_v15.xlsx"
fileslist = [f for f in listdir(mypath) if isfile(join(mypath, f))]
xlsx_files = []
for file in fileslist:
    if file.endswith('.xlsx'):
        xlsx_files.append(file)
main_df = pd.DataFrame()

In [11]:
for file in xlsx_files:
    file_path = join(mypath, file)
    wb = openpyxl.load_workbook(file_path, data_only = True)
    ws = wb['Payment Term History']
    ws = wb.active # have it active
    ws.cell(1, 1).value = user_text  # Replace the content in A1
    df = pd.DataFrame(ws.values)
    wb.save(file_path)  # Save the workbook after making changes
    wb.close()  # Ensure the workbook is closed after processing

In [12]:
# Refresh data
for file in xlsx_files:
    file_path = join(mypath, file)
    xlapp = win32com.client.DispatchEx("Excel.Application")
    wb = xlapp.Workbooks.Open(file_path)
    wb.RefreshAll()
    xlapp.CalculateUntilAsyncQueriesDone()
    wb.Save()
    wb.Close() 
    xlapp.Quit()

In [5]:
# def df_from_excel(file_path):
#     app = xlwings.App(visible=False)
#     book = app.books.open(file_path)
#     book.save()
#     book.close()
#     app.kill()
#   return pd.read_excel(path)
# df = df_from_excel(file_path)

In [6]:
# first try
for file in xlsx_files:
    wb = load_workbook(r"C:\Pam_card\summary\file\_Data_CutOff_v15.xlsx")
    ws = wb['DATA']
    ws = wb.active
    df = pd.DataFrame(ws.values)

In [13]:
for file in xlsx_files:
    file_path = join(mypath, file)
    wb = openpyxl.load_workbook(file_path, data_only=True)
    ws = wb['Payment Term History']
    try:
        df = pd.read_excel(file_path, sheet_name='Payment Term History',  skiprows=4, usecols=lambda x: 'Unnamed: 0' not in x)
    except ValueError as e:
        continue  
    df = df.dropna(subset=['pam_code'])

    if 'choose' in df['pam_code'].values:
        first_choose_index = df[df['pam_code'] == 'choose'].index[0]
        df = df.loc[:first_choose_index - 1]
    
    main_df = pd.concat([main_df, df], ignore_index=True)
main_df.to_excel(join(mypath, 'merge_g.xlsx'), index=False)